In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
from scipy.optimize import curve_fit

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [188]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_path = filedialog.askopenfilename(multiple=True) # initialdir = dir_string
# print(file_path)

time       = []
T_aux_avg  = []
r2_v2_rlim      = []

for i,j in enumerate(file_path):
    print(j)
    with load(j) as data:
        time.append(data['time'])
        T_aux_avg.append(data['temp'])
        r2_v2_rlim.append(data['r2_v2_rlim'])

/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0512_DC03_RF03.npz


In [129]:
myRF = 24

for k in range(3):
    print(r2_v2_rlim[myRF][k+6][50]*1000) # [condition][dim][time]


R = r2_v2_rlim[myRF][6][50]*1000
L = r2_v2_rlim[myRF][8][50]*1000
aws = 0
print((R+(1.48*aws/2))/(L+1.48*aws/2))

88.4032051741599
91.99671408043952
70.20884466891401
1.2591462741061412


In [7]:
def tanh_to10(x,x0,K,theta):
    return 10**( K*(1+tanh(theta*(x-x0))) )
def tanh_plus_to10(x,x0,K,theta,a,b):
    return 10**( K*(1+tanh(theta*(x-x0))) + a*(x)+b )

def gen_logi_to10(x,A,K,C,B,M,nu):
    return 10**( A + ( K-A ) / ( C+exp(-B*(x-M)) )**(nu) )

def func2_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+(t-t0)**2)**D + B )
def func2abs_to10(t, t0, A, B,  C, D):
    return 10**( A*(t-t0)/(C+abs(t-t0))**D + B )
def func2ter_to10(t, t0, A, B, D):
    return 10**( A*(t-t0)**(2*D) / (1+(t-t0)**2)**D + B )
#def func2bis_to10(t, t0, A, B,  C, D):
#    tau = t-t0
#    return 10**( (A-B)*(tau) / (C+(tau**2)**(D/2))**(1/D) + B )
def func3_to10(t, t0, A, B,  C):
    return 10**( A*(t-t0) / (C+(t-t0)**2) + B )
def func4_to10(t, t0, A, B, C):
    return 10**( A*(t-t0) / (C+abs(t-t0)) + B )
def func5_to10(t, t0, A, B, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B )
def func6_to10(t, t0, A, C, k):
    return 10**( A*(t-t0) / (C+abs(t-t0)**k)**(1/k) )
def func7_to10(t, t0, A, B, k):
    return 10**( A*(t-t0) / (1+abs(t-t0)**k)**(1/k) + B)

In [8]:
def func2(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+(t-t0)**2)**D + B
def func2abs(t, t0, A, B,  C, D):
    return A*(t-t0)/(C+abs(t-t0))**D + B
def func5(t, t0, A, B, C, k):
    return A*(t-t0) / (C+abs(t-t0)**k)**(1/k) + B

In [139]:
def func_lin(t,a,b):
    return a*t+b
def func_exp(x,a,b):
    return b*exp(-a*x)
def func_pow(t,A,B):
    return t**A * 10**B

In [172]:
cm = pylab.get_cmap('tab10')

In [13]:
for i,j in enumerate(file_path):
    print(i,j)

0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
6 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC11_RF11.npz
7 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC12_RF12.npz
8 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC13_RF13.npz
9 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_

In [191]:
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
Urf = array([0, 0, 0, 0, 0, 55.35111, 57.40115, 59.45119, 61.50124, 63.55128, 65.60132, 67.65136, 69.70140, 71.75144,0, 0, 0, 0, 0, 64.6, 62.5, 62.83, 63.03, 63.24, 63.44])
q = array([0, 0, 0, 0, 0, 0.54, 0.56,  0.58,  0.6, 0.62,  0.64,  0.66,  0.68,  0.7, 0, 0, 0, 0, 0, 0.63, 0.61, 0.612, 0.614, 0.616, 0.618])
Udc = array([0, 0, 0, 0, 0, 11.10109, 12.0829, 13.12915, 14.24497, 15.43671, 16.71181, 18.07917, 19.54956, 21.13623, 0, 0, 0, 0, 0, 16.0, 14.8, 14.95, 15.07, 15.19, 15.31])

beta_guess = 0.423
a = 0
for k in range(250):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)

omega_z_2 = ( 2*pi*100e3 )**2 * array(Udc)
omega_x_2 = ( beta_guess*Omega/2 )**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2
print(sqrt(omega_r_2[my_try])/2/pi)

fs = 1/diff(x)[1]

401073.4060130965


In [18]:
import matplotlib.ticker as ticker

In [169]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')

to_fit = 0 #11 19 8
temp_to_end = 100

my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
print('>',to_fit,file_path[to_fit])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

index_100K = argmin(abs(y_filt-temp_to_end))
end_fit = index_100K

t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)



p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log10(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print('func5',popt)

figname = 'testfitinlin0'
figure(figname,clear='True')
#xlim(0.8,6)
ax2 = subplot(111)
ax2.grid()
ax2.semilogy(x*1e3, y ,label=r'$T$',color='C0',lw=0.3)
ax2.semilogy(x*1e3, y_filt ,label=r'$T$ filtered',color='C1',ls=':',lw=3)
ax2.vlines(x[end_fit]*1e3,1e-2,150,color='k')

y_fit5 = func5(x[start_for_fit:end_fit],*popt)
ax2.semilogy(x[start_for_fit:end_fit]*1e3,10**y_fit5,color='r',ls='--',label=r'$\psi$',lw=2)

nticks = 9
maj_loc = ticker.LogLocator(numticks=nticks)
min_loc = ticker.LogLocator(subs='all', numticks=nticks)
ax2.yaxis.set_major_locator(maj_loc)
ax2.yaxis.set_minor_locator(min_loc)

ax2.set_xlabel('t [ms]')
ax2.set_ylabel('T [K]')
# ax2.set_xlim(-0.3,5.6)
# ax2.set_ylim(2e-3,300)
tight_layout()
# savefig(figname+'.eps',dpi=300)

file to analyze
> 0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz


KeyboardInterrupt: 

In [197]:
# Fit all curves
# Smooth data temperature
# guess first parameter set
# fit smoothed data up to threshold temperature
# much more consistent than below

popt_smooth = np.zeros((len(file_path),5))
temp_to_end = [100, 100, 75, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100, 100, 100, 100, 100] # Temperature to stop the fit
# 100, 100, 100, 100, 100, 100, 100, 100, 100   75, 75, 75, 75, 75, 75, 75, 75, 75
start_for_fit = 200
T_crit = 0.5
print('file to analyze')
not_fitted = 0
for to_fit in range(len(file_path[:-2])):
    j = file_path[to_fit]
# to_fit = 30
# j = file_path[to_fit]
    if to_fit!=0 and to_fit!=9 and to_fit!=10 and to_fit!=11:
    # retrive time and temperature from the curve
        print('>',to_fit,j)
        my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
        x  = time[to_fit]-1e-3
        y  = T_aux_avg[to_fit]

        # Smoothing signal with zero delay shift fit
        max_f_r = sqrt(omega_r_2[my_try])/2/pi
        b, a = signal.butter(8,  max_f_r/fs*2.01)
        y_filt = signal.filtfilt(b, a, y, padlen=150)

        # Fit parameters first guesses
        t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
        B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
        A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

        # select index of temp where stop the fit
        # I stop the fit when T reaches a given T
        index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
        end_fit = index_100K

        # set initial guess of fit parameters
        p0 = [t0_first_guess,A0,B0,0.01,0.3]
        try:
            # proper fit of the smoothed curve limiter up to threshold T
            popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                                          log10(y_filt[start_for_fit:end_fit]),
                                   p0, maxfev= 25000,
                                   bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
            print(popt)
            popt_smooth[to_fit,:] = popt
        except RuntimeError:
            print('Optimal parameters not found: Number of calls to function has reached maxfev')
            not_fitted += 1
    else:
        print('>',to_fit,j)
        print('Data skipped')
        not_fitted += 1
    #     if to_fit == 28:
    #         tm.sleep(5)
    #     else:
    #         tm.sleep(0.5)
print('total amount of non fitted curves',not_fitted)

file to analyze
> 0 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC05_RF05.npz
Data skipped
> 1 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC06_RF06.npz
[ 0.18492212  3.98391909 -0.01898171  0.05204516  0.11902239]
> 2 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC07_RF07.npz
[ 0.06242862  3.62601072 -0.01101076  0.0445645   0.15040419]
> 3 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC08_RF08.npz
[ 0.02173411  3.60442781 -0.0693643   0.04231391  0.15852348]
> 4 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC09_RF09.npz
[ 0.00737913  3.59901386 -0.14109287  0.03953713  0.17363417]
> 5 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N0256_DC10_RF10.npz
[ 0.00308311  2.78613363 -0.22488433  0.01926918  0.25635418]
> 6 /home/adrien/Documents/Programmes

In [199]:
start_for_fit = 200
T_crit = 0.5
print('file to analyze')
not_fitted = 0
# 29 and 30
to_fit = 34
temp_to_end = [100, 100, 75, 50, 50, 50, 50, 50, 50,
               75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
               100, 100, 100, 100, 100, 100, 100, 100, 75, 150, 100, 100, 100, 100, 100]
j = file_path[to_fit]
# retrive time and temperature from the curve
print('>',to_fit,j)
my_try = int(file_path[to_fit][file_path[to_fit].find('DC')+2:file_path[to_fit].find('DC')+4])
x  = time[to_fit]-1e-3
y  = T_aux_avg[to_fit]

# Smoothing signal with zero delay shift fit
max_f_r = sqrt(omega_r_2[my_try])/2/pi
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

# Fit parameters first guesses
t0_first_guess = x[where(min(abs(y_filt-T_crit))==abs(y_filt-T_crit))][0] # 2.5e-3
B0 = log10(T_crit) # log10( y[where(min(abs(y-T_crit))==abs(y-T_crit))][0] )
A0 = log10(max(y_filt[start_for_fit:end_fit]) - B0)

# select index of temp where stop the fit
# I stop the fit when T reaches a given T
index_100K = argmin(abs(y_filt-temp_to_end[to_fit]))
end_fit = index_100K

# set initial guess of fit parameters
p0 = [t0_first_guess,A0,B0,0.01,0.3]

# proper fit of the smoothed curve limiter up to threshold T
popt, pcov = curve_fit(func5, x[start_for_fit:end_fit],
                              log10(y_filt[start_for_fit:end_fit]),
                       p0, maxfev= 25000,
                       bounds=((-np.inf,-np.inf,-np.inf,0,0),(np.inf,np.inf,2,1.01,1)))
print(popt)
popt_smooth[to_fit,:] = popt

file to analyze
> 34 /home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/Time_and_temp_RFHEAT_N1024_DC24_RF24.npz
[0.00303696 2.25705018 0.00976478 0.00367462 0.4602244 ]


In [ ]:
figure('coucou2')
plot(array(popt_rework[1])[:,0])
array(popt_rework[1])[:,0]

In [200]:
popt_rework = [popt_smooth[1:9],
              popt_smooth[12:20], # 7:15
              popt_smooth[20:]]
#  3     4     5     6     7     8     9     10    11    12    13
# 31.0, 42.0, 55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8
#             0.54  0.56  0.58  0.6   0.62  0.64  0.66  0.68  0.7
Urf_rework = [[57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8],
             [57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8], # 57.4, 59.5, 
             [55.4, 57.4, 59.5, 61.5, 63.6, 65.6, 67.7, 69.7, 71.8, 64.6, 62.5, 62.83, 63.03, 63.24, 63.44]]
q_rework = [[0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7],
             [0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7 ], # 57.4, 59.5, 
             [0.54, 0.56,  0.58,  0.6,   0.62,  0.64,  0.66,  0.68,  0.7, 0.63, 0.61, 0.612, 0.614, 0.616, 0.618]]
N_rework = [256, 512, 1024]

for k,l in enumerate(N_rework):
    print(f'fit_params_{l:04d}ions')
    np.savez( f'fit_params_{l:04d}ions', fit_param=popt_rework[k], Urf=Urf_rework[k], N=N_rework[k])

# func5(t, t0, A, B, C, k)
# with load('/home/adrien/Documents/Programmes/Python/THESE/RF_Temp_Fit/fit_params_0256ions.npz') as data:
#     a = data['N']
# print(a)

fit_params_0256ions
fit_params_0512ions
fit_params_1024ions


In [201]:
figure('fit_parameters',clear=True)
ax1 = subplot(151)
for k in range(3):
    print(k)
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,0]*1e3-1,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
    ax2 = subplot(152)
    ax2.plot(Urf_rework[k],10**array(popt_rework[k])[:,1],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax2.set_xlabel('$U_{RF}$ [V]')
    ax2.set_ylabel('$10^A$ [K]')
    ax2.grid()
    ax2.legend(title='N',fontsize=12)
    ax3 = subplot(153)
    ax3.plot(Urf_rework[k],10**array(popt_rework[k])[:,2],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax3.set_xlabel('$U_{RF}$ [V]')
    ax3.set_ylabel('$10^B$ [K]')
    ax3.grid()
    ax3.legend(title='N',fontsize=12)
    ax4 = subplot(154)
    ax4.plot(Urf_rework[k],array(popt_rework[k])[:,3]*1e3,color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax4.set_xlabel('$U_{RF}$ [V]')
    ax4.set_ylabel('$C$ [ms]')
    ax4.grid()
    ax4.legend(title='N',fontsize=12)
    ax5 = subplot(155)
    ax5.plot(Urf_rework[k],array(popt_rework[k])[:,4],color=cm(k/10),ls=':',marker='P',label=N_rework[k])
    ax5.set_xlabel('$U_{RF}$ [V]')
    ax5.set_ylabel('$k$ [1]')
    ax5.grid()
    ax5.legend(title='N',fontsize=12)
tight_layout()

0
1
2


In [202]:
C = []
D = []
x_forfit = [linspace(0.555,0.71,200),linspace(0.555,0.71,200),linspace(0.535,0.71,200)]
figname = 'fit_parameter_t0_log'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.loglog(q_rework[k],array(popt_rework[k])[:,0]*1e3,
                 color=cm(k/10),ls='',marker='P',ms=10,mew=1,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, log10(array(q_rework[k])),log10(array(popt_rework[k])[:,0]),maxfev= 5000)
    print(k,popt)
    C.append(10**popt[1])
    D.append(popt[0])
    ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
                 ls='--',color=cm(k/10))
    ax1.set_xlabel('$q_{x}$')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=18)
# popt, pcov = curve_fit(func_lin, hstack([log10(array(q_rework[k][:5])),log10(array(q_rework[k][6:]))]),
#                        hstack([log10(array(popt_rework[k])[:,0][:5]),log10(array(popt_rework[k])[:,0][6:])])
#                        ,maxfev= 5000)
# ax1.loglog(x_forfit[k],func_pow(x_forfit[k],popt[0],popt[1])*1e3,
#                  ls='--',color=cm(k/10))
ax1.set_xticks([0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7])
ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax1.grid(visible=True, which='major', color='0.2', linestyle='-')
ax1.grid(visible=True, which='minor', color='0.7', linestyle='--')
tight_layout()
savefig(figname+'.eps',dpi=300)
    
fig = figure('fit_parameter_t0_bis',clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),color=cm(k/4),
             ls=':',marker='P',ms=10,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),(array(popt_rework[k])[:,0]*1e3)**(-1/2),maxfev= 15000)
    ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),color=cm(k/3))
    ax1.set_xlabel('$q_x$')
    ax1.set_ylabel('$t_0^{-1/2}$ [ms$^{-1/2}$]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
tight_layout()

0 [-27.44310085  -7.73212055]
1 [-24.39301074  -7.34163438]
2 [-20.37905235  -6.79732172]


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [203]:
C,D

([1.853017211046684e-08, 4.5537126206658615e-08, 1.594697394309545e-07],
 [-27.443100851149545, -24.393010737102532, -20.37905235420479])

In [117]:
fig = figure('fit_parameter_B',clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(Urf_rework[k],array(popt_rework[k])[:,2]*1e3,color=cm(k/5),ls=':',marker='P',ms=10,mew=1,label=N_rework[k])
    ax1.set_xlabel('$U_{RF}$ [V]')
    ax1.set_ylabel('$t_0$ [ms]')
    ax1.grid()
    ax1.legend(title='N',fontsize=12)
tight_layout()
    
figname = 'fit_parameter_B_bis'
fig = figure(figname,clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = subplot(111)
for k in range(3):
    ax1.plot(array(q_rework[k]),10**array(popt_rework[k])[:,2],
             color=cm(k/10),ls='',marker='P',ms=10,label=N_rework[k])
    popt, pcov = curve_fit(func_lin, array(q_rework[k]),(10**array(popt_rework[k])[:,2]),maxfev= 15000)
    ax1.plot(array(q_rework[k]),func_lin(array(q_rework[k]),popt[0],popt[1]),
             color=cm(k/10),ls='--')
    ax1.set_xlabel('$q_x$')
    ax1.set_ylabel('$B=T(t=t_0)$ [K]')
    ax1.grid()
    ax1.legend(title='N',fontsize=18)
tight_layout()
savefig(figname+'.png',dpi=300)

# Derivate study

In [ ]:
c_light = 299792458
h_pl = 6.62607015*1e-34
h_pl_bar = h_pl/2/pi
lam_397 = 396.84620*1e-9
k=(2*pi)/lam_397
gamma_SP = 21.57*1e6
tau = 1/gamma_SP
Omega = 2.0e6*2*pi
tau_RF = 1/(Omega/(2*pi))

In [ ]:
to_fit = 24
x  = time[to_fit]-1e-3
dx = x[3]-x[2]
y  = T_aux_avg[to_fit]
b, a = signal.butter(8,  max_f_r/fs*1.01)
y_filt = signal.filtfilt(b, a, y, padlen=150)

dT_dt = -1/tau*sqrt(m_Ca*y_filt/kb)*h_pl_bar*k/m_Ca

fig = figure('temperature derivate',clear=True)
fig.set_size_inches(11.7, 8.3)
ax1 = fig.add_subplot(111)

# ax1.semilogy(x*1e3,y_filt)
# ax1.plot(x[:-1]*1e3,diff(y_filt))
ax1.plot(y_filt[:-1],diff(y_filt)*1e3)
ax1.plot(y_filt,-dT_dt*tau_RF*1e3)

# ax1.set_xlim(0.7,1.2)
# ax1.set_ylim(-0.02,0.3*1e6)

ax1.set_xlim(-2,50)
ax1.set_ylim(-1,3.3*1e3)
ax1.set_xlabel('T [K]')
ax1.set_ylabel('T [mK/RF period]')

ax1.grid()
tight_layout()

# From fluo-variations_optimal-temp.ipynb

# Functions used to describe ions

$\texttt{MB}$ is the Maxwell-Boltzmann distribution. It provides the probability to find an atom with a given velocity in a gas with temperature T.

$\texttt{pfl_dop} = \rho_{ee}$ is the atomic ray profile with Doppler effect. It provides the probability of excitation given Rabi frequency, detuning, lambda and velocity of atom. It is considered equal to the excited population, i.e the proportion of excited atoms.

$ \texttt{MB} = \sqrt{\frac{m}{2\pi k_BT}}\exp{-\frac{mv^2}{2k_BT}}$

$\texttt{pfl_dop} = \rho_{ee} = \frac{0.25\Omega_R^2}{0.5\Omega_R^2 + 0.25\Gamma^2 + \Delta^2} = \frac{A/2}{A+B + \Delta^2}$

In [ ]:
# Doppler profile
def pfl_dop(v, delta, k, Rab, Gam):
    return .25*Rab**2/(0.5*Rab**2+.25*Gam**2+(delta-k*v)**2)

# Maxwell-Boltzmann distribution
def MB(v, T):
    kb = 1.38e-23
    m_Ca = 40*1.66e-27
    return (m_Ca/(np.pi*2*kb*T))**(1/2) * np.exp(-m_Ca*v**2/(2*kb*T)) # **1 car vitesse par rapport au laser compte seulement (1D)

# Probabilities product
# For a range of velocities,
# compute the prob_fluo
# for a given T, delta, Rab, Gam
def prob_fluo(vmin, vmax, nv, T, delta, k, Rab, Gam):
    nu = k*np.linspace(vmin, vmax, nv)
    return nu, pfl_dop(-nu/k, delta, k, Rab, Gam)*MB(nu/k, T)

def T_lim(delta):
    return -0.5*hbar*Gam**2*(1+(2*delta/Gam)**2) / (4*delta)/kb

In [ ]:
# Laser parameters
k = 15826663 # m^-1
Gam = 21570000.0 *2*pi
delta = Gam
lam = 397e-9 # m
I = 170 # W/m²

# Fluorescence

For a single ion the fluorescence $F \propto \rho$. For an ensemble of ions, the fluorescence F is related to the product of both probabilities $ F \propto \texttt{MB} \times \texttt{pfl_dop} = \texttt{prob_fluo}$. Maxwell-Boltzmann is not necessarily the best to describe the cold crystal ensemble but lets use it. This product is still a probability, lets call it $P$. $dP/dT = 0$ gives us the temperature for which the fluo is maximum, $d^2P/dT^2 = 0$ gives us the temperature for which the fluorescence variation  $dP/dT$ is maximum.

### Analytically with $\rho$

$\frac{d\rho}{dv} = \frac{A}{2}\frac{2\sqrt{z}}{(C+z)^2}, z=(\delta-v)^2, C = A + B$

$\frac{d^2\rho}{dv^2} = \frac{f}{g} = [-2(C+z)^2 + 8z(C+z)]\frac{A}{2(\dots)^4}$

The idea is to search for the setting where the variation of fluorescence is the highest for a given input in energy. The energy introduced by the GiantMolecule is brought under the form of thermal energy. Thus we want to set the cloud in a configuration with the best variation of fluorescence for a variation in temperature. There are two ways of achieving this. Considering infinitesimal differences, this requires to compute $\frac{d^2\rho}{dv^2} = 0$.

$\frac{d^2\rho}{dv^2} = 0 \Rightarrow f = 0 \Rightarrow z = \frac{1}{3}C \Rightarrow \delta - v = \pm \sqrt{\frac{C}{3}} = \pm \sqrt{\frac{A+B}{3}} $

Considering the thermal velocity $v = \sqrt{2k_BT/m}$ it is possible to rewrite $v = \delta - z$  as $2k_BT/m = (\delta - z)^2$ then $T = \frac{m(\delta - z)^2}{(2k_B)}\frac{1}{k^2}$ 

The above computation is done just below

In [ ]:
# Plot the MBxpfl-dop vs Temperature

N = N_rework[2]

Tmin = .0005
Tmax = 10
vmin = sqrt(2*kb*Tmin/m_Ca)
vmax = sqrt(2*kb*Tmax/m_Ca)
nu = -k*np.linspace(vmin,vmax,1000)
dv = mean(diff(nu))/k

Rab = Gam # or np.sqrt(3*lam**3*Gam/(4*np.pi**2*hbar*c) * I) 

vmoins = -20
vplus = 20
nv = 1000
dv = (vplus-vmoins)/nv

pfl0 = []
pfl05 = []
pfl1 = []
pfl2 = []

Temps = linspace(0.005,10,1000)
for T in Temps:
    pfl0.append( sum(prob_fluo(vmoins, vplus, nv, T, 0, k, Rab, Gam)[1])*dv*Gam * N )
    pfl05.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam/2, k, Rab, Gam)[1])*dv*Gam * N )
    pfl1.append( sum(prob_fluo(vmoins, vplus, nv, T, -Gam, k, Rab, Gam)[1])*dv*Gam * N )
    pfl2.append( sum(prob_fluo(vmoins, vplus, nv, T, Gam, k, Rab, Gam)[1])*dv*Gam * N )
# pfl3 = pfl_dop(nu/k, +3*Gam, k, Rab, Gam)

fig = plt.figure('MBxDop prof fluo tot',clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)

ax.plot(Temps, array(pfl0)*1e-9, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot(Temps, array(pfl05)*1e-9, ':',color='xkcd:black',label=r'$\delta = -\Gamma/2$')
ax.plot(Temps, array(pfl2)*1e-9, '-',color='xkcd:blue',label=r'$\delta = \Gamma$')
ax.plot(Temps, array(pfl1)*1e-9, '--',color='xkcd:red',label=r'$\delta = -\Gamma$')
# ax.plot((nu/k)**2*m_Ca/2/kb, pfl3, '-',color='xkcd:black',label=r'$\delta = \Gamma$')

ax.hlines((pfl2[0]-sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
ax.hlines((pfl2[0]+sqrt(pfl2[0]))*1e-9,0,1,color='xkcd:red',ls=':',lw = 3)
# ax.hlines(pfl0[0]-sqrt(pfl0[0]),0,0.1,color='xkcd:green',ls=':',lw = 3)
# ax.hlines(pfl05[0]-sqrt(pfl05[0]),0,0.1,color='xkcd:blue',ls=':',lw = 3)


ax.set_xlabel(r'T [K]')
ax.set_ylabel(r'Fluorescence $F$ [kcts/µs]')
ax.set_xlim(-0.02,2.22)
# ax.set_ylim(12,46)

ax.grid()
ax.legend(title='detuning',loc=1,fontsize=15)

plt.tight_layout()
# subplots_adjust(top=0.96,
# bottom=0.135,
# left=0.115,
# right=0.965,
# hspace=0.2,
# wspace=0.2)

In [ ]:
fig = plt.figure('d/dt MBxDop prof fluo tot',clear='True')
fig.set_size_inches(11.7, 8.3)
ax = fig.add_subplot(111)

ax.plot(Temps[:-1], diff(array(pfl0))*1e-9, '-',color='xkcd:green',label=r'$\delta = 0$')
ax.plot(Temps[:-1], diff(array(pfl05))*1e-9, ':',color='xkcd:black',label=r'$\delta = -\Gamma/2$')
ax.plot(Temps[:-1], diff(array(pfl1))*1e-9, '--',color='xkcd:red',label=r'$\delta = -\Gamma$')
ax.grid()
ax.set_xlim(0,1)
ax.legend(title='detuning',loc=1,fontsize=15)